# Chapter 19: Token Standards

---

Tokens are the lifeblood of blockchain ecosystems. They represent assets—whether currencies, collectibles, or rights—and enable a vast array of decentralized applications. But for tokens to be interoperable, they must follow standards. This chapter explores the most important token standards on Ethereum: ERC-20 (fungible tokens), ERC-721 (non-fungible tokens), ERC-1155 (multi-token), and several others that power modern DeFi and NFTs. You'll learn how they work, why they matter, and how to implement them.

---

## 19.1 ERC-20: Fungible Tokens

ERC-20 is the most widely adopted token standard. It defines a common interface for fungible tokens—tokens where every unit is identical and interchangeable, like currencies or loyalty points.

### 19.1.1 Standard Functions and Events

The ERC-20 standard, defined in [EIP-20](https://eips.ethereum.org/EIPS/eip-20), requires the following functions and events:

**Required Functions:**
```solidity
function totalSupply() external view returns (uint256);
function balanceOf(address account) external view returns (uint256);
function transfer(address to, uint256 amount) external returns (bool);
function allowance(address owner, address spender) external view returns (uint256);
function approve(address spender, uint256 amount) external returns (bool);
function transferFrom(address from, address to, uint256 amount) external returns (bool);
```

**Optional (but commonly implemented):**
```solidity
function name() external view returns (string memory);
function symbol() external view returns (string memory);
function decimals() external view returns (uint8);
```

**Required Events:**
```solidity
event Transfer(address indexed from, address indexed to, uint256 value);
event Approval(address indexed owner, address indexed spender, uint256 value);
```

### 19.1.2 Implementation Details

Let's build a minimal ERC-20 token from scratch to understand the mechanics.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

contract MinimalERC20 {
    // State variables
    string public name;
    string public symbol;
    uint8 public decimals;
    uint256 public totalSupply;

    mapping(address => uint256) public balanceOf;
    mapping(address => mapping(address => uint256)) public allowance;

    // Events
    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);

    constructor(string memory _name, string memory _symbol, uint8 _decimals, uint256 _initialSupply) {
        name = _name;
        symbol = _symbol;
        decimals = _decimals;
        totalSupply = _initialSupply * 10 ** decimals;
        balanceOf[msg.sender] = totalSupply;
        emit Transfer(address(0), msg.sender, totalSupply);
    }

    function transfer(address to, uint256 amount) external returns (bool) {
        require(to != address(0), "ERC20: transfer to zero address");
        require(balanceOf[msg.sender] >= amount, "ERC20: insufficient balance");

        balanceOf[msg.sender] -= amount;
        balanceOf[to] += amount;
        emit Transfer(msg.sender, to, amount);
        return true;
    }

    function approve(address spender, uint256 amount) external returns (bool) {
        require(spender != address(0), "ERC20: approve to zero address");

        allowance[msg.sender][spender] = amount;
        emit Approval(msg.sender, spender, amount);
        return true;
    }

    function transferFrom(address from, address to, uint256 amount) external returns (bool) {
        require(from != address(0), "ERC20: transfer from zero address");
        require(to != address(0), "ERC20: transfer to zero address");
        require(balanceOf[from] >= amount, "ERC20: insufficient balance");
        require(allowance[from][msg.sender] >= amount, "ERC20: insufficient allowance");

        allowance[from][msg.sender] -= amount;
        balanceOf[from] -= amount;
        balanceOf[to] += amount;
        emit Transfer(from, to, amount);
        return true;
    }
}
```

**Explanation:**
- `balanceOf` maps addresses to token balances.
- `allowance` is a nested mapping: `owner => (spender => amount)`, tracking how much a spender can withdraw from an owner.
- `transfer` moves tokens from the caller to another address.
- `approve` sets the allowance for a spender.
- `transferFrom` moves tokens from `from` to `to` using the caller's allowance.

### 19.1.3 Extensions (Mintable, Burnable, Pausable)

In practice, you'll often need additional functionality. OpenZeppelin provides standard extensions.

**Mintable:**
```solidity
import "@openzeppelin/contracts/token/ERC20/ERC20.sol";
import "@openzeppelin/contracts/access/Ownable.sol";

contract MyToken is ERC20, Ownable {
    constructor() ERC20("MyToken", "MTK") {
        _mint(msg.sender, 1000000 * 10 ** decimals());
    }

    function mint(address to, uint256 amount) public onlyOwner {
        _mint(to, amount);
    }
}
```

**Burnable:**
```solidity
import "@openzeppelin/contracts/token/ERC20/extensions/ERC20Burnable.sol";

contract MyBurnableToken is ERC20Burnable, ERC20 {
    constructor() ERC20("MyToken", "MTK") {}
}
```

**Pausable:** Allows owner to pause transfers (useful for emergencies).
```solidity
import "@openzeppelin/contracts/token/ERC20/extensions/ERC20Pausable.sol";

contract MyPausableToken is ERC20Pausable, ERC20, Ownable {
    constructor() ERC20("MyToken", "MTK") {}
    function pause() public onlyOwner { _pause(); }
    function unpause() public onlyOwner { _unpause(); }
}
```

### 19.1.4 Complete Implementation

Here's a production-ready ERC-20 using OpenZeppelin:

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC20/ERC20.sol";
import "@openzeppelin/contracts/access/Ownable.sol";
import "@openzeppelin/contracts/token/ERC20/extensions/ERC20Burnable.sol";
import "@openzeppelin/contracts/token/ERC20/extensions/ERC20Pausable.sol";

contract CompleteERC20 is ERC20, Ownable, ERC20Burnable, ERC20Pausable {
    constructor(
        string memory name,
        string memory symbol,
        uint256 initialSupply
    ) ERC20(name, symbol) {
        _mint(msg.sender, initialSupply * 10 ** decimals());
    }

    function pause() public onlyOwner {
        _pause();
    }

    function unpause() public onlyOwner {
        _unpause();
    }

    function mint(address to, uint256 amount) public onlyOwner {
        _mint(to, amount);
    }

    // Override required functions for multiple inheritance
    function _beforeTokenTransfer(address from, address to, uint256 amount)
        internal
        override(ERC20, ERC20Pausable)
    {
        super._beforeTokenTransfer(from, to, amount);
    }
}
```

---

## 19.2 ERC-721: Non-Fungible Tokens (NFTs)

ERC-721 defines the interface for non-fungible tokens—each token is unique. Used for art, collectibles, and digital ownership.

### 19.2.1 Standard Functions and Events

**Required Functions:**
```solidity
function balanceOf(address owner) external view returns (uint256 balance);
function ownerOf(uint256 tokenId) external view returns (address owner);
function safeTransferFrom(address from, address to, uint256 tokenId, bytes calldata data) external;
function safeTransferFrom(address from, address to, uint256 tokenId) external;
function transferFrom(address from, address to, uint256 tokenId) external;
function approve(address to, uint256 tokenId) external;
function setApprovalForAll(address operator, bool approved) external;
function getApproved(uint256 tokenId) external view returns (address operator);
function isApprovedForAll(address owner, address operator) external view returns (bool);
```

**Required Events:**
```solidity
event Transfer(address indexed from, address indexed to, uint256 indexed tokenId);
event Approval(address indexed owner, address indexed approved, uint256 indexed tokenId);
event ApprovalForAll(address indexed owner, address indexed operator, bool approved);
```

**Optional Metadata Extension:**
```solidity
function name() external view returns (string memory);
function symbol() external view returns (string memory);
function tokenURI(uint256 tokenId) external view returns (string memory);
```

### 19.2.2 Metadata Structure

ERC-721 tokens often have metadata stored off-chain (usually on IPFS). The `tokenURI` function returns a URI pointing to a JSON file conforming to the [ERC-721 Metadata JSON Schema](https://eips.ethereum.org/EIPS/eip-721).

**Example metadata:**
```json
{
    "name": "My NFT #1",
    "description": "A unique digital collectible",
    "image": "ipfs://QmZbH...",
    "attributes": [
        {
            "trait_type": "Background",
            "value": "Blue"
        },
        {
            "trait_type": "Eyes",
            "value": "Green"
        }
    ]
}
```

### 19.2.3 Minting and Transferring NFTs

Let's build a minimal ERC-721 token.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC721/ERC721.sol";
import "@openzeppelin/contracts/access/Ownable.sol";
import "@openzeppelin/contracts/utils/Counters.sol";

contract MyNFT is ERC721, Ownable {
    using Counters for Counters.Counter;
    Counters.Counter private _tokenIdCounter;

    string private _baseTokenURI;

    constructor(string memory baseURI) ERC721("MyNFT", "MNFT") {
        _baseTokenURI = baseURI;
    }

    function _baseURI() internal view override returns (string memory) {
        return _baseTokenURI;
    }

    function safeMint(address to) public onlyOwner {
        uint256 tokenId = _tokenIdCounter.current();
        _tokenIdCounter.increment();
        _safeMint(to, tokenId);
    }

    function setBaseURI(string memory baseURI) public onlyOwner {
        _baseTokenURI = baseURI;
    }

    function totalSupply() public view returns (uint256) {
        return _tokenIdCounter.current();
    }
}
```

**Explanation:**
- `Counters` provides a safe way to increment token IDs.
- `_baseURI` is overridden to return our base URI; `tokenURI` will concatenate `baseURI + tokenId`.
- `safeMint` uses `_safeMint` which checks that the recipient can receive ERC-721 tokens (important for contracts).

### 19.2.4 Complete Implementation

A more complete NFT contract with minting to specific addresses and setting token URIs individually:

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC721/extensions/ERC721URIStorage.sol";
import "@openzeppelin/contracts/access/Ownable.sol";
import "@openzeppelin/contracts/utils/Counters.sol";

contract AdvancedNFT is ERC721URIStorage, Ownable {
    using Counters for Counters.Counter;
    Counters.Counter private _tokenIds;

    constructor() ERC721("AdvancedNFT", "ANFT") {}

    function mintNFT(address recipient, string memory tokenURI)
        public
        onlyOwner
        returns (uint256)
    {
        _tokenIds.increment();
        uint256 newItemId = _tokenIds.current();
        _mint(recipient, newItemId);
        _setTokenURI(newItemId, tokenURI);
        return newItemId;
    }

    function totalSupply() public view returns (uint256) {
        return _tokenIds.current();
    }
}
```

---

## 19.3 ERC-1155: Multi-Token Standard

ERC-1155 is a efficient standard that can represent both fungible and non-fungible tokens in a single contract. It's ideal for gaming, where you might have both currencies and unique items.

### 19.3.1 Fungible and Non-Fungible in One

ERC-1155 uses a single contract to manage multiple token types. Each token type has an `id`. For fungible tokens, multiple units of the same id exist; for NFTs, each id has only one unit (or is tracked individually).

**Key functions:**
```solidity
function balanceOf(address account, uint256 id) external view returns (uint256);
function balanceOfBatch(address[] calldata accounts, uint256[] calldata ids) external view returns (uint256[] memory);
function setApprovalForAll(address operator, bool approved) external;
function isApprovedForAll(address account, address operator) external view returns (bool);
function safeTransferFrom(address from, address to, uint256 id, uint256 amount, bytes calldata data) external;
function safeBatchTransferFrom(address from, address to, uint256[] calldata ids, uint256[] calldata amounts, bytes calldata data) external;
```

### 19.3.2 Batch Operations

One of the biggest advantages of ERC-1155 is **batch transfers** and **batch balance checks**, which save gas.

**Batch mint example:**
```solidity
function mintBatch(address to, uint256[] memory ids, uint256[] memory amounts, bytes memory data) public {
    _mintBatch(to, ids, amounts, data);
}
```

### 19.3.3 Use Cases

- **Gaming**: One contract for gold (fungible), swords (fungible items with durability?), and unique hero NFTs.
- **Marketplaces**: Can handle multiple token types in a single contract.
- **Fractionalized NFTs**: Represent partial ownership of an NFT with fungible tokens.

### 19.3.4 Complete Implementation

Using OpenZeppelin:

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC1155/ERC1155.sol";
import "@openzeppelin/contracts/access/Ownable.sol";
import "@openzeppelin/contracts/utils/Strings.sol";

contract MyMultiToken is ERC1155, Ownable {
    using Strings for uint256;

    string public name;
    string public symbol;

    constructor(string memory _name, string memory _symbol, string memory baseURI) ERC1155(baseURI) {
        name = _name;
        symbol = _symbol;
    }

    function mint(address account, uint256 id, uint256 amount, bytes memory data)
        public
        onlyOwner
    {
        _mint(account, id, amount, data);
    }

    function mintBatch(address to, uint256[] memory ids, uint256[] memory amounts, bytes memory data)
        public
        onlyOwner
    {
        _mintBatch(to, ids, amounts, data);
    }

    function setURI(string memory newuri) public onlyOwner {
        _setURI(newuri);
    }

    // Optional: override to return custom URI per token
    function uri(uint256 tokenId) public view override returns (string memory) {
        return string(abi.encodePacked(super.uri(tokenId), tokenId.toString(), ".json"));
    }
}
```

---

## 19.4 Other Important Standards

Beyond the core token standards, several other ERCs have become crucial in modern DeFi and NFT ecosystems.

### 19.4.1 ERC-777: Advanced Token Standard

ERC-777 aims to improve upon ERC-20 by adding hooks and more advanced features. It's backward compatible with ERC-20 but introduces:

- **Operators**: Can send tokens on behalf of users.
- **Hooks**: Contracts can react to receiving tokens (like `tokensReceived`), enabling more complex logic.
- **Granularity**: Tokens can be sent with data.

However, due to reentrancy risks and complexity, ERC-777 has been less adopted than ERC-20. It's worth knowing but not recommended for new projects unless specific hooks are needed.

### 19.4.2 ERC-4626: Tokenized Vaults

ERC-4626 standardizes yield-bearing vaults. It defines a common interface for tokenized vaults that accept deposits of an underlying asset (e.g., DAI) and return shares representing a claim on the vault's assets plus yield.

**Key functions:**
```solidity
function asset() external view returns (address);
function totalAssets() external view returns (uint256);
function convertToShares(uint256 assets) external view returns (uint256);
function convertToAssets(uint256 shares) external view returns (uint256);
function maxDeposit(address receiver) external view returns (uint256);
function previewDeposit(uint256 assets) external view returns (uint256);
function deposit(uint256 assets, address receiver) external returns (uint256);
function mint(uint256 shares, address receiver) external returns (uint256);
function withdraw(uint256 assets, address receiver, address owner) external returns (uint256);
function redeem(uint256 shares, address receiver, address owner) external returns (uint256);
```

This standard enables composability: vaults can be plugged into DeFi aggregators, lending protocols, and more.

**Example:**
```solidity
import "@openzeppelin/contracts/token/ERC20/extensions/ERC4626.sol";

contract MyVault is ERC4626 {
    constructor(IERC20 _asset) ERC4626(_asset) {}
    // Additional yield strategies can be implemented
}
```

### 19.4.3 ERC-2612: Permit (Gasless Approvals)

ERC-2612 extends ERC-20 with a `permit` function that allows approvals via signatures. This enables **gasless transactions**: a user can sign a message, and a relayer can submit it, paying gas on their behalf.

**How it works:**
- User signs a message containing `(owner, spender, value, nonce, deadline)`.
- Any relayer calls `permit` with the signature.
- The contract verifies the signature and sets the allowance.
- Then the relayer can call `transferFrom` to move tokens.

This is crucial for DeFi where users might not have ETH for gas, or for smoother UX.

**Implementation snippet:**
```solidity
function permit(
    address owner,
    address spender,
    uint256 value,
    uint256 deadline,
    uint8 v,
    bytes32 r,
    bytes32 s
) external {
    require(block.timestamp <= deadline, "Permit: expired");
    bytes32 structHash = keccak256(abi.encode(PERMIT_TYPEHASH, owner, spender, value, nonces[owner]++, deadline));
    bytes32 hash = _hashTypedDataV4(structHash);
    address signer = ECDSA.recover(hash, v, r, s);
    require(signer == owner, "Permit: invalid signature");
    _approve(owner, spender, value);
}
```

### 19.4.4 ERC-1967: Proxy Storage Slots

ERC-1967 is not a token standard but a **proxy storage standard**. It defines storage slots for proxy contracts to store the implementation address and other upgrade-related data. It's used by UUPS and transparent proxies to prevent storage collisions.

**Key slots:**
- `0x360894a13ba1a3210667c828492db98dca3e2076cc3735a920a3ca505d382bbc` = `bytes32(uint256(keccak256('eip1967.proxy.implementation')) - 1)`
- `0xb53127684a568b3173ae13b9f8a6016e243e63b6e8ee1178d6a717850b5d6103` = admin slot

Developers rarely need to interact with these directly unless writing proxy contracts, but understanding them is useful for debugging.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 19 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Token standards ensure interoperability across applications.   │
│                                                                 │
│  ERC-20 (Fungible Tokens):                                     │
│    • balanceOf, transfer, approve, transferFrom                │
│    • Extensions: mintable, burnable, pausable                  │
│    • OpenZeppelin provides audited implementations              │
│                                                                 │
│  ERC-721 (Non-Fungible Tokens):                                │
│    • Each token is unique (tokenId)                            │
│    • ownerOf, safeTransferFrom                                 │
│    • Metadata stored off-chain (tokenURI)                      │
│                                                                 │
│  ERC-1155 (Multi-Token):                                       │
│    • Single contract for multiple token types                  │
│    • Batch operations for efficiency                           │
│    • Ideal for gaming and mixed assets                         │
│                                                                 │
│  Other Important Standards:                                    │
│    • ERC-777: Advanced token with hooks (use cautiously)       │
│    • ERC-4626: Yield-bearing vaults                            │
│    • ERC-2612: Permit for gasless approvals                    │
│    • ERC-1967: Proxy storage slots for upgrades                │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 20 – Zero-Knowledge Proofs. We'll explore the mathematics and applications of ZK proofs, including zk-SNARKs and zk-STARKs, and how they're used for privacy and scalability.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='18. oracles_and_external_data.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='20. zero_knowledge_proofs.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
